# Gas Mixing Calculation Using AGA8 GERG-2008

This example demonstrates how to calculate the resulting gas composition when mixing multiple streams with different compositions, pressures, and temperatures.

## Scenario

Three gas streams are involved:
- **Gas 1** and **Gas 2** are produced into a single mixing point
- A reactor removes **Gas 3** from this mixture
- **Stream 4** (output) = Gas 1 + Gas 2 - Gas 3

Each stream is measured by an ultrasonic flowmeter (volume flowrate Q in m³/h). We'll use GERG-2008 to calculate density, then mass flowrate, and finally the composition of Stream 4.

## 1. Import Required Libraries

In [1]:
import pvtlib
import pandas as pd
import numpy as np

## 2. Define Input Parameters

Input data for the three gas streams:

In [2]:
# Volume flowrates [m3/h]
Q_gas1 = 5000
Q_gas2 = 3000
Q_gas3 = 500

# Pressures [bara]
P_gas1 = 150
P_gas2 = 130
P_gas3 = 30

# Temperatures [°C]
T_gas1 = 50
T_gas2 = 40
T_gas3 = 30

# Gas compositions [mole %]
comp_gas1 = {
    'N2': 1,
    'CO2': 2,
    'C1': 90,
    'C2': 6,
    'C3': 0.9,
    'iC4': 0.05,
    'nC4': 0.05
}

comp_gas2 = {
    'N2': 10,
    'CO2': 5,
    'C1': 81,
    'C2': 4
}

comp_gas3 = {
    'N2': 10,
    'CO2': 60,
    'C1': 30
}

# Display input data
input_df = pd.DataFrame({
    'Gas 1': [Q_gas1, P_gas1, T_gas1],
    'Gas 2': [Q_gas2, P_gas2, T_gas2],
    'Gas 3': [Q_gas3, P_gas3, T_gas3]
}, index=['Q [m³/h]', 'P [bara]', 'T [°C]'])

print("Input Parameters:")
print(input_df)
print("\nGas Compositions [mole %]:")
print(f"Gas 1: {comp_gas1}")
print(f"Gas 2: {comp_gas2}")
print(f"Gas 3: {comp_gas3}")

Input Parameters:
          Gas 1  Gas 2  Gas 3
Q [m³/h]   5000   3000    500
P [bara]    150    130     30
T [°C]       50     40     30

Gas Compositions [mole %]:
Gas 1: {'N2': 1, 'CO2': 2, 'C1': 90, 'C2': 6, 'C3': 0.9, 'iC4': 0.05, 'nC4': 0.05}
Gas 2: {'N2': 10, 'CO2': 5, 'C1': 81, 'C2': 4}
Gas 3: {'N2': 10, 'CO2': 60, 'C1': 30}


## 3. Calculate Gas Properties Using GERG-2008

Use the AGA8 GERG-2008 equation of state to calculate density and other properties for each stream.

In [3]:
# Initialize GERG-2008
gerg = pvtlib.AGA8('GERG-2008')

# Calculate properties for Gas 1
gerg_gas1 = gerg.calculate_from_PT(
    composition=comp_gas1,
    pressure=P_gas1,
    temperature=T_gas1,
    pressure_unit='bara',
    temperature_unit='C'
)

# Calculate properties for Gas 2
gerg_gas2 = gerg.calculate_from_PT(
    composition=comp_gas2,
    pressure=P_gas2,
    temperature=T_gas2,
    pressure_unit='bara',
    temperature_unit='C'
)

# Calculate properties for Gas 3
gerg_gas3 = gerg.calculate_from_PT(
    composition=comp_gas3,
    pressure=P_gas3,
    temperature=T_gas3,
    pressure_unit='bara',
    temperature_unit='C'
)

# Display densities
print("Calculated Densities using GERG-2008:")
print(f"Gas 1: ρ = {gerg_gas1['rho']:.3f} kg/m³")
print(f"Gas 2: ρ = {gerg_gas2['rho']:.3f} kg/m³")
print(f"Gas 3: ρ = {gerg_gas3['rho']:.3f} kg/m³")

Calculated Densities using GERG-2008:
Gas 1: ρ = 118.400 kg/m³
Gas 2: ρ = 111.650 kg/m³
Gas 3: ρ = 44.622 kg/m³


## 4. Calculate Mass Flowrates

Calculate mass flowrate from volume flowrate and density:
$$\dot{m} = Q \times \rho$$

where:
- $\dot{m}$ = mass flowrate [kg/h]
- $Q$ = volumetric flowrate [m³/h]
- $\rho$ = density [kg/m³]

In [4]:
# Calculate mass flowrates [kg/h]
mass_flow_gas1 = Q_gas1 * gerg_gas1['rho']
mass_flow_gas2 = Q_gas2 * gerg_gas2['rho']
mass_flow_gas3 = Q_gas3 * gerg_gas3['rho']

# Display results
mass_flow_df = pd.DataFrame({
    'Gas 1': [Q_gas1, gerg_gas1['rho'], mass_flow_gas1],
    'Gas 2': [Q_gas2, gerg_gas2['rho'], mass_flow_gas2],
    'Gas 3': [Q_gas3, gerg_gas3['rho'], mass_flow_gas3]
}, index=['Q [m³/h]', 'ρ [kg/m³]', 'Mass Flow [kg/h]'])

print("Mass Flowrate Calculations:")
print(mass_flow_df.round(2))

Mass Flowrate Calculations:
                      Gas 1      Gas 2     Gas 3
Q [m³/h]            5000.00    3000.00    500.00
ρ [kg/m³]            118.40     111.65     44.62
Mass Flow [kg/h]  592000.58  334948.81  22310.89


## 5. Calculate Stream 4 Composition

Use the AGA8 mix function to calculate the resulting composition:

**Stream 4 = Gas 1 + Gas 2 - Gas 3**

The mix function takes a list of compositions and their corresponding masses (negative mass for subtraction).

In [5]:
# Mix the streams: Gas1 + Gas2 - Gas3
# Note: Gas 3 has negative mass (subtraction)
stream4 = gerg.mix(
    compositions=[comp_gas1, comp_gas2, comp_gas3],
    mix_weights=[mass_flow_gas1, mass_flow_gas2, -mass_flow_gas3]
)

# Display results
print("Stream 4 Results:")
print(f"\nTotal Mass Flowrate: {stream4['total_mass']:.2f} kg/h")
print(f"\nComposition [mole %]:")
for component, mole_percent in sorted(stream4['composition'].items(), 
                                      key=lambda x: x[1], reverse=True):
    print(f"  {component:>4s}: {mole_percent:6.3f}%")

Stream 4 Results:

Total Mass Flowrate: 904638.50 kg/h

Composition [mole %]:
    C1: 87.644%
    C2:  5.380%
    N2:  4.026%
   CO2:  2.286%
    C3:  0.597%
   iC4:  0.033%
   nC4:  0.033%


## 5.1 Calculate Stream 4 Volume Flowrate and Velocity

Calculate the volume flowrate and velocity of Stream 4 in a 20" FC10A pipe (ID = 408 mm).

**Note:** For Stream 4, we assume the gas is at Gas 1 conditions (P = 150 bara, T = 50°C).

In [6]:
# Assume Stream 4 is at Gas 1 conditions
P_stream4 = P_gas1  # bara
T_stream4 = T_gas1  # °C

# Calculate GERG properties for Stream 4 composition
gerg_stream4 = gerg.calculate_from_PT(
    composition=stream4['composition'],
    pressure=P_stream4,
    temperature=T_stream4,
    pressure_unit='bara',
    temperature_unit='C'
)

# Calculate volume flowrate [m³/h]
# Q = mass_flow / density
Q_stream4 = stream4['total_mass'] / gerg_stream4['rho']

# Pipe geometry
pipe_ID = 408  # mm
pipe_ID_m = pipe_ID / 1000  # Convert to meters

# Calculate cross-sectional area [m²]
pipe_area = np.pi * (pipe_ID_m / 2)**2

# Calculate velocity [m/s]
# v = Q / A, but Q is in m³/h, so convert to m³/s
Q_stream4_m3s = Q_stream4 / 3600  # Convert m³/h to m³/s
velocity_stream4 = Q_stream4_m3s / pipe_area

# Display results
print("Stream 4 Flow Calculations:")
print("-" * 60)
print(f"Pipe: 20\" FC10A, ID = {pipe_ID} mm")
print(f"Conditions: P = {P_stream4} bara, T = {T_stream4} °C")
print(f"Density: ρ = {gerg_stream4['rho']:.3f} kg/m³")
print(f"\nVolume Flowrate: Q = {Q_stream4:.2f} m³/h ({Q_stream4_m3s:.4f} m³/s)")
print(f"Velocity: v = {velocity_stream4:.2f} m/s")

Stream 4 Flow Calculations:
------------------------------------------------------------
Pipe: 20" FC10A, ID = 408 mm
Conditions: P = 150 bara, T = 50 °C
Density: ρ = 118.378 kg/m³

Volume Flowrate: Q = 7641.94 m³/h (2.1228 m³/s)
Velocity: v = 16.24 m/s


## 6. Summary

Create a comprehensive summary table showing all streams and their properties.

In [7]:
# Create comprehensive summary
summary_df = pd.DataFrame({
    'Gas 1': [Q_gas1, P_gas1, T_gas1, gerg_gas1['rho'], mass_flow_gas1, '-'],
    'Gas 2': [Q_gas2, P_gas2, T_gas2, gerg_gas2['rho'], mass_flow_gas2, '-'],
    'Gas 3': [-Q_gas3, P_gas3, T_gas3, gerg_gas3['rho'], -mass_flow_gas3, '-'],
    'Stream 4': [Q_stream4, P_stream4, T_stream4, gerg_stream4['rho'], stream4['total_mass'], velocity_stream4]
}, index=['Q [m³/h]', 'P [bara]', 'T [°C]', 'ρ [kg/m³]', 'Mass Flow [kg/h]', 'Velocity [m/s]'])

print("=" * 70)
print("SUMMARY - GAS MIXING CALCULATION")
print("=" * 70)
print("\nStream Properties:")
print(summary_df.round(2))

print("\n\nStream 4 Composition (sorted by mole %):")
print("-" * 40)
for component, mole_percent in sorted(stream4['composition'].items(), 
                                      key=lambda x: x[1], reverse=True):
    print(f"  {component:>4s}: {mole_percent:7.3f}%")
print("-" * 40)
print(f"  Total: {sum(stream4['composition'].values()):7.3f}%")

print("\n" + "=" * 70)
print(f"Stream 4 = Gas 1 + Gas 2 - Gas 3")
print(f"         = {mass_flow_gas1:.2f} + {mass_flow_gas2:.2f} - {mass_flow_gas3:.2f}")
print(f"         = {stream4['total_mass']:.2f} kg/h")
print(f"\nStream 4 Flow (20\" FC10A pipe, ID = 408 mm):")
print(f"         Q = {Q_stream4:.2f} m³/h")
print(f"         v = {velocity_stream4:.2f} m/s")
print("=" * 70)

SUMMARY - GAS MIXING CALCULATION

Stream Properties:
                         Gas 1          Gas 2         Gas 3   Stream 4
Q [m³/h]                  5000           3000          -500    7641.94
P [bara]                   150            130            30     150.00
T [°C]                      50             40            30      50.00
ρ [kg/m³]           118.400116     111.649603      44.62178     118.38
Mass Flow [kg/h]  592000.58238  334948.809611 -22310.889932  904638.50
Velocity [m/s]               -              -             -      16.24


Stream 4 Composition (sorted by mole %):
----------------------------------------
    C1:  87.644%
    C2:   5.380%
    N2:   4.026%
   CO2:   2.286%
    C3:   0.597%
   iC4:   0.033%
   nC4:   0.033%
----------------------------------------
  Total: 100.000%

Stream 4 = Gas 1 + Gas 2 - Gas 3
         = 592000.58 + 334948.81 - 22310.89
         = 904638.50 kg/h

Stream 4 Flow (20" FC10A pipe, ID = 408 mm):
         Q = 7641.94 m³/h
         v =